In [8]:
from operator import add
import os
import math


def word_freq(file_name):
    data = sc.textFile(file_name)
    data_freq = data.flatMap(lambda x : x.split(' ')) \
          .map(lambda x: x.lower()) \
          .filter(lambda x: x not in [',', '.', '/', '!', '?', '-'] and x!='' ) \
          .map(lambda x: (x,1)) \
          .reduceByKey(add)
    return data_freq

def idf(folder):
    files = os.listdir(folder)
    files_data = []
    for i in files:
        try:
            file_name = os.path.join(folder, i)
            data = sc.textFile(file_name)
            data_file = data.flatMap(lambda x: x.split(' '))\
                            .map(lambda x: x.lower())\
                            .map(lambda x: (x, 1))\
                            .reduceByKey(add)\
                            .map(lambda x: (x[0], 1))
            files_data.append(data_file)
        except Exception as ex :
            print(i)
            print(ex)
        
    union_data = sc.union(files_data)
    doc_data = union_data.reduceByKey(add).map(lambda x: (x[0], math.log10(len(files)/x[1])))
    
    return doc_data


def tf(file_name):
    data = sc.textFile(file_name)
    total_word = data.flatMap(lambda x : x.split(' ')) \
          .map(lambda x: x.lower()) \
          .filter(lambda x: x not in [',', '.', '/', '!', '?', '-'] and x!='' ) \
          .map(lambda x: 1).reduce(add)
    tf_data = word_freq(file_name).map(lambda x: (x[0], x[1]/total_word))
    return tf_data
    

def main(folder):
    idf_values = idf(folder)
    files = os.listdir(folder)
    data = []
    for i in files:
        try:
            file_name = os.path.join(folder, i)
            tf_data = tf(file_name)
            data_file = tf_data.join(idf_values)\
                                .map(lambda x: (x[0], i, x[1][0]*x[1][1]))
            data.append(data_file)
        except Exception as ex:
            print(i)
            print(ex)
    tfidf = sc.union(data)
    return tfidf
        

In [19]:
b = tf('/home/prates/nltk_data/corpora/mac_morpho/ag94ag02.txt')

In [59]:

data = idf('/home/prates/nltk_data/corpora/mac_morpho/')

In [9]:
data = main('/home/prates/teste_tf-idf/')

In [10]:
data.take(100)


[('cidade_n', 'ag94ab12.txt', 0.0),
 ('caf�_n', 'ag94ab12.txt', 0.0),
 ('oscilou_v', 'ag94ab12.txt', 5.437680557514111e-05),
 ('participa_v', 'ag94ab12.txt', 5.437680557514111e-05),
 ('chuvas_n', 'ag94ab12.txt', 0.0),
 ('oferta_n', 'ag94ab12.txt', 0.0),
 ('�ltimas_adj', 'ag94ab12.txt', 0.0),
 ('pre�os_n', 'ag94ab12.txt', 0.0),
 ('13,6_num', 'ag94ab12.txt', 5.437680557514111e-05),
 ('que_ks', 'ag94ab12.txt', 0.0),
 ('a_prep', 'ag94ab12.txt', 0.0),
 ('estados_n', 'ag94ab12.txt', 0.0),
 ('trabalhando_v', 'ag94ab12.txt', 5.437680557514111e-05),
 ('toledo_nprop', 'ag94ab12.txt', 0.00010875361115028222),
 ('nacional_adj', 'ag94ab12.txt', 0.0),
 ('ano_n', 'ag94ab12.txt', 0.0),
 ('segundo_prep', 'ag94ab12.txt', 0.0),
 ('assessoria_nprop', 'ag94ab12.txt', 5.437680557514111e-05),
 ('meados_n', 'ag94ab12.txt', 5.437680557514111e-05),
 ('alguns_proadj', 'ag94ab12.txt', 0.0),
 ('v�o_vaux', 'ag94ab12.txt', 0.0),
 ('10_num', 'ag94ab12.txt', 0.0),
 ('excelente_adj', 'ag94ab12.txt', 5.437680557514111e-

In [105]:
c = sc.union([r1,r2])

In [106]:
print(c.collect())

[1, 2, 3, 4, 1, 2, 3, 4]
